<a href="https://colab.research.google.com/github/databykarinna/pengantar-data-sains/blob/main/Mini_Project_week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mini Project**

In [32]:
# Import library

import numpy as np
import pandas as pd

a.	Bangun data panel “harga saham” sederhana dengan unit id ∈ {𝐴,𝐵,𝐶}, rentang 30 hari (date), dan variabel numerik price.

In [33]:
# Membuat dimensi panel

## Daftar unit saham (cross-section)
ids = ["A", "B", "C"]

## Membuat 30 tanggal berurutan mulai 1 September 2026 (frekuensi harian)
dates = pd.date_range("2026-09-01", periods=30, freq="D")

## Membuat seluruh kombinasi antara id dan tanggal (struktur panel long)
df_panel = (
    pd.MultiIndex.from_product([ids, dates], names=["id", "date"])
    .to_frame(index=False)   # Mengubah MultiIndex menjadi DataFrame biasa
)

# Membuat harga saham (random walk)

df_panel["price"] = (
    df_panel.groupby("id")   # Mengelompokkan berdasarkan id (tiap saham dihitung terpisah)
    .apply(lambda x:
           np.round(
               # Membuat harga awal sekitar 76 lalu ditambah pergerakan acak kumulatif
               76 + np.cumsum(np.random.normal(0.2, 1, len(x))),
               2  # Membulatkan ke 2 angka desimal
           )
    )
    .explode()        # Meratakan hasil agar sesuai dengan panjang DataFrame
    .astype(float)    # Memastikan tipe data numerik (float)
    .values           # Mengambil nilai array untuk dimasukkan ke kolom
)

# Menampilkan tabel untuk memastikan struktur sudah benar
df_panel

/tmp/ipython-input-1760543086.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x:


,id,date,price
0,A,2026-09-01,76.60
1,A,2026-09-02,77.26
2,A,2026-09-03,77.99
3,A,2026-09-04,77.58
4,A,2026-09-05,78.28
...,...,...,...
85,C,2026-09-26,85.10
86,C,2026-09-27,86.51
87,C,2026-09-28,88.32
88,C,2026-09-29,87.31


b.	Pastikan data berada pada format long dengan struktur kolom minimal: (id, date,  price).
Jika data masih wide, lakukan konversi wide → long.


In [34]:
# Misal datanya wide
panel_wide = df_panel.pivot(index="date", columns="id", values="price").reset_index()
panel_wide

id,date,A,B,C
0,2026-09-01,76.60,76.15,76.69
1,2026-09-02,77.26,75.08,75.87
2,2026-09-03,77.99,74.45,75.97
3,2026-09-04,77.58,74.53,77.29
4,2026-09-05,78.28,74.84,76.88
5,2026-09-06,77.03,75.78,77.07
6,2026-09-07,77.57,76.38,76.67
7,2026-09-08,77.11,76.42,77.41
8,2026-09-09,78.87,76.42,76.97
9,2026-09-10,78.07,76.61,77.03


In [35]:
# Ubah menjadi data long
panel_long = (
    panel_wide
    .melt(id_vars="date", var_name="id", value_name="price")
    .sort_values(["id", "date"])
    .reset_index(drop=True)
)

panel_long

,date,id,price
0,2026-09-01,A,76.60
1,2026-09-02,A,77.26
2,2026-09-03,A,77.99
3,2026-09-04,A,77.58
4,2026-09-05,A,78.28
...,...,...,...
85,2026-09-26,C,85.10
86,2026-09-27,C,86.51
87,2026-09-28,C,88.32
88,2026-09-29,C,87.31


c.	Hitung log-return per id:
$$
r_{t} = \log\!\left(\text{price}_{t}\right) - \log\!\left(\text{price}_{t-1}\right)
$$
(dihitung terpisah untuk setiap id).

In [36]:
# Mengurutkan data berdasarkan id dan tanggal agar urutan waktu benar
# .copy() digunakan untuk menghindari warning saat modifikasi kolom
df_panel = df_panel.sort_values(["id", "date"]).copy()

# Menghitung log natural dari harga
df_panel["log_price"] = np.log(df_panel["price"])

# Menghitung log-return per id
# diff() untuk menghitung selisih antara nilai sekarang dan nilai sebelumnya
# groupby("id") untuk memastikan perhitungan dilakukan terpisah untuk tiap saham
df_panel["log_return"] = (
    df_panel.groupby("id")["log_price"]
    .diff()
)

df_panel

,id,date,price,log_price,log_return
0,A,2026-09-01,76.60,4.338597,NaN
1,A,2026-09-02,77.26,4.347176,0.008579
2,A,2026-09-03,77.99,4.356581,0.009404
3,A,2026-09-04,77.58,4.351310,-0.005271
4,A,2026-09-05,78.28,4.360292,0.008982
...,...,...,...,...,...
85,C,2026-09-26,85.10,4.443827,0.026675
86,C,2026-09-27,86.51,4.460260,0.016433
87,C,2026-09-28,88.32,4.480967,0.020707
88,C,2026-09-29,87.31,4.469465,-0.011502


In [37]:
# Memastikan
df_panel[df_panel["log_return"].isna()]

,id,date,price,log_price,log_return
0,A,2026-09-01,76.60,4.338597,NaN
30,B,2026-09-01,76.15,4.332705,NaN
60,C,2026-09-01,76.69,4.339771,NaN


d.	Bangun dua fitur untuk klasifikasi dini dari setiap seri (per id), misalnya:


*   rolling mean 5-hari pada price, dan
*   rolling volatility 5-hari (simpangan baku) pada log-return.






In [38]:
# Membuat fitur rolling mean 5-hari pada harga (price)

df_panel["roll_mean_5"] = (
    df_panel.groupby("id")["price"]   # Mengelompokkan data per saham (per id)
    .transform(
        lambda x: x.rolling(window=5).mean()
        # rolling(window=5) → mengambil rata-rata 5 hari terakhir
        # mean() → menghitung rata-ratanya
    )
)

df_panel


,id,date,price,log_price,log_return,roll_mean_5
0,A,2026-09-01,76.60,4.338597,NaN,NaN
1,A,2026-09-02,77.26,4.347176,0.008579,NaN
2,A,2026-09-03,77.99,4.356581,0.009404,NaN
3,A,2026-09-04,77.58,4.351310,-0.005271,NaN
4,A,2026-09-05,78.28,4.360292,0.008982,77.542
...,...,...,...,...,...,...
85,C,2026-09-26,85.10,4.443827,0.026675,82.208
86,C,2026-09-27,86.51,4.460260,0.016433,83.500
87,C,2026-09-28,88.32,4.480967,0.020707,84.982
88,C,2026-09-29,87.31,4.469465,-0.011502,86.020


In [39]:
# Memastikan
df_panel[df_panel["roll_mean_5"].isna()]

,id,date,price,log_price,log_return,roll_mean_5
0,A,2026-09-01,76.60,4.338597,NaN,NaN
1,A,2026-09-02,77.26,4.347176,0.008579,NaN
2,A,2026-09-03,77.99,4.356581,0.009404,NaN
3,A,2026-09-04,77.58,4.351310,-0.005271,NaN
30,B,2026-09-01,76.15,4.332705,NaN,NaN
31,B,2026-09-02,75.08,4.318554,-0.014151,NaN
32,B,2026-09-03,74.45,4.310128,-0.008426,NaN
33,B,2026-09-04,74.53,4.311202,0.001074,NaN
60,C,2026-09-01,76.69,4.339771,NaN,NaN
61,C,2026-09-02,75.87,4.329021,-0.010750,NaN


In [40]:
# Membuat fitur rolling volatility 5-hari berdasarkan log-return

df_panel["roll_vol_5"] = (
    df_panel.groupby("id")["log_return"]   # Mengelompokkan data per saham
    .transform(
        lambda x: x.rolling(window=5).std()
        # rolling(window=5) → mengambil 5 log-return terakhir
        # std() → menghitung simpangan baku (volatilitas)
    )
)
df_panel

,id,date,price,log_price,log_return,roll_mean_5,roll_vol_5
0,A,2026-09-01,76.60,4.338597,NaN,NaN,NaN
1,A,2026-09-02,77.26,4.347176,0.008579,NaN,NaN
2,A,2026-09-03,77.99,4.356581,0.009404,NaN,NaN
3,A,2026-09-04,77.58,4.351310,-0.005271,NaN,NaN
4,A,2026-09-05,78.28,4.360292,0.008982,77.542,NaN
...,...,...,...,...,...,...,...
85,C,2026-09-26,85.10,4.443827,0.026675,82.208,0.010041
86,C,2026-09-27,86.51,4.460260,0.016433,83.500,0.006928
87,C,2026-09-28,88.32,4.480967,0.020707,84.982,0.006622
88,C,2026-09-29,87.31,4.469465,-0.011502,86.020,0.014765


In [41]:
# Memastikan
df_panel[df_panel["roll_vol_5"].isna()]

,id,date,price,log_price,log_return,roll_mean_5,roll_vol_5
0,A,2026-09-01,76.60,4.338597,NaN,NaN,NaN
1,A,2026-09-02,77.26,4.347176,0.008579,NaN,NaN
2,A,2026-09-03,77.99,4.356581,0.009404,NaN,NaN
3,A,2026-09-04,77.58,4.351310,-0.005271,NaN,NaN
4,A,2026-09-05,78.28,4.360292,0.008982,77.542,NaN
30,B,2026-09-01,76.15,4.332705,NaN,NaN,NaN
31,B,2026-09-02,75.08,4.318554,-0.014151,NaN,NaN
32,B,2026-09-03,74.45,4.310128,-0.008426,NaN,NaN
33,B,2026-09-04,74.53,4.311202,0.001074,NaN,NaN
34,B,2026-09-05,74.84,4.315353,0.004151,75.010,NaN
